In [53]:
import datasets
import glob

In [54]:
train_dataset = {"text":[], "label":[]}

In [55]:
math_file_paths = glob.glob("../data/examination_paper_markdown" + '/' + "*.md")

for file_path in math_file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        train_dataset["text"].append(f.read())
        train_dataset["label"].append(1)

In [56]:
math_file_paths = glob.glob("../data/not_examination_paper_markdown" + '/' + "*.md")

for file_path in math_file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        train_dataset["text"].append(f.read())
        train_dataset["label"].append(0)

In [57]:
train_dataset = datasets.Dataset.from_dict(train_dataset)

In [58]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 2773
})

In [59]:
test_dataset = {"text":[], "label":[]}

In [60]:
math_file_paths = glob.glob("../data/test_examination_paper_markdown" + '/' + "*.md")

for file_path in math_file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        test_dataset["text"].append(f.read())
        test_dataset["label"].append(1)

In [61]:
math_file_paths = glob.glob("../data/test_not_examination_paper_markdown" + '/' + "*.md")

for file_path in math_file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        test_dataset["text"].append(f.read())
        test_dataset["label"].append(0)

In [62]:
test_dataset = datasets.Dataset.from_dict(test_dataset)

In [63]:
datadict = datasets.DatasetDict({
    "train":train_dataset,
    "test":test_dataset
})

In [64]:
datadict.save_to_disk("./dataset")

Saving the dataset (0/1 shards):   0%|          | 0/2773 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/392 [00:00<?, ? examples/s]